# <center>Simulation of Lennard-John potential </center>

#### This code is presentation of behaviour partitions in Lennard-John potential. We observe position and velocity of single partition and measure features of whole system like temporary temperature or pressure. We can also track the process of  phase transition from gas to solid state and measure characteristic average  distance between particles to identify physical state of our system.  

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from tqdm import tqdm

In [2]:
!pip install tqdm

In [3]:
class czastka:
    
    def __init__(self, radius, pos,vel=[0,0],vel_past=[0,0],vel_futher=[0,0]):
        self.rad = radius
        self.r = pos
        self.v = vel
        self.v_minus = vel_past
        self.v_plus = vel_futher 
        self.momentum = vel*m
        

def Sila(r1,r2):
        r_vect=r1-r2
        
        if r_vect[0]>boxsize2:
            r_vect[0]=r_vect[0]-boxsize
        elif r_vect[0]<-boxsize2:
            r_vect[0]=r_vect[0]+boxsize

        if r_vect[1]>boxsize2:
             r_vect[1]=r_vect[1]-boxsize
        elif r_vect[1]<-boxsize2:
            r_vect[1]=r_vect[1]+boxsize
        
        dlugosc=np.linalg.norm(r_vect)
        
        if(dlugosc>=2.5):
            sila=[0,0]
        else:
            sila=48*eps/(sigma**2)*((sigma/dlugosc)**14-0.5*(sigma/dlugosc)**8)*r_vect
        
        return sila
        

def Temperatura(particles):
    N = len(particles)
    
    T = (1./(boltzman_const*N)) * sum([np.linalg.norm(particles[i].momentum)**2/m for i in range(N)])
    
    return T

def Skalowanie_temperatury(particles):
    '''Skaluje prędkości cząstek aby ich średnia temperatura była równa zadanej'''
    
    for p in particles:
        p.momentum *=np.sqrt(temp_termostat/temperatura_poczatkowa)
        
        
def Suma_sil(particles):
    forces =[]
    for i, particle1 in enumerate(particles):#i to indeks do elementu particle1
        force=np.zeros(2)
        for particle2 in particles[:i]+particles[i+1:]:
            force+=Sila(particle1.r,particle2.r)
        
        forces.append(force)
    return forces


def frog(particles,sila):
    for i,p in enumerate(particles):
        p.v=p.v_minus+(sila[i]/m)*(deltat/2)
        p.momentum=p.v*m
    
    T=Temperatura(particles)
    eta=np.sqrt(temp_termostat/T)
    
    for i,p in enumerate(particles):
        p.v=p.v_minus+(sila[i]/m)*(deltat/2)
        p.momentum=p.v*m
        p.v_plus=(2*eta-1)*p.v_minus + eta*(sila[i]/m)*deltat
        p.r+=p.v_plus*deltat
        p.v_minus=p.v_plus
        
        

def Suma_sil_wektor(particles):
    forces =[]
    odleglosci=[]
    iloczyn_sila_polozenie=0
    for i, particle1 in enumerate(particles):#i to indeks do elementu particle1
        force=[]
        odleglosc=[]
        for particle2 in particles[:i]+particles[i+1:]:
            s=Sila(particle1.r,particle2.r)
            force.append(s)
            r=particle1.r-particle2.r
            odleglosc.append(r)
            iloczyn_sila_polozenie+=sum(s*r)
        forces.append(force)
        odleglosci.append(odleglosc)
    return np.array(forces),np.array(odleglosci),iloczyn_sila_polozenie      
        
        
        
def Cisnienie(particles):
    N=len(particles)
    F_cisnienia,R_cisnienia,iloczyn = Suma_sil_wektor(particles)
    T= Temperatura(particles)
    V=boxsize*boxsize
    
#     for i in range(N):
#         iloczyn_sila_polozenie+=np.sum(np.dot(F_cisnienia[i],particles[i].r))
    
        
        
    cisnienie = N*T/V +(1/2*V)*iloczyn
    return cisnienie


def Energia_kinetyczna(particles):
    Energia=0
    for p in particles:
        Energia+=1/2.*m*np.dot(p.v,p.v)
    return Energia

def Energia_potencjalna(particles):
    energy=0
    for i, particle1 in enumerate(particles):#i to indeks do elementu particle1
        for particle2 in particles[:i]+particles[i+1:]:
            energy+=LJPotential(particle1.r,particle2.r)
        
    return energy

def LJPotential(r1,r2):
        r_vect=r1-r2
        
        if r_vect[0]>boxsize2:
            r_vect[0]=r_vect[0]-boxsize
        elif r_vect[0]<-boxsize2:
            r_vect[0]=r_vect[0]+boxsize

        if r_vect[1]>boxsize2:
             r_vect[1]=r_vect[1]-boxsize
        elif r_vect[1]<-boxsize2:
            r_vect[1]=r_vect[1]+boxsize
        


        dlugosc=np.linalg.norm(r_vect)
        
        potential=4*eps*((sigma/dlugosc)**12-(sigma/dlugosc)**6)
        return potential

In [4]:
#Warunki początkowe oraz stałe
boxsize = 10.0
boxsize2= boxsize/2
eps = 1.0
sigma = 1.0
promien = 0.5
deltat = 0.0001
temp_termostat=2.0#zamarzanie dla 0.3
nx = 7
ny = 7  # wielkosc siatki do generowania cząstek
delta = 1.1
t_max = 10
particleNumber = nx*ny
m = 1
boltzman_const=1.
density=(particleNumber/(boxsize*boxsize))*sigma**2

In [8]:
density

0.49

In [9]:
particles=[]
for i in range(nx):  # generowanie punktów sieci,inicjalizacja polożeń i prędkości
    for j in range(ny):
        polozenie = np.array([i * delta + boxsize / nx, j * delta + boxsize / ny])
        predkosc = np.array([(np.random.random() - 1. / 2), (np.random.random() - 1. / 2)])
        p0=predkosc*m
        particles.append(czastka(promien,polozenie,predkosc,vel_past=[0,0],vel_futher=[0,0]))

    

In [11]:
temperatura_poczatkowa=Temperatura(particles)
print(temperatura_poczatkowa)
Skalowanie_temperatury(particles)

#ustawianie czasu ujemnego
suma=Suma_sil(particles)
for i, p in enumerate(particles):
    p.v=p.momentum/m
    p.v_minus=p.v - 0.5*suma[i]/m*deltat 

print(Temperatura(particles))

2.0
2.0


In [12]:
def Distance_from_particle(particles,choosen_particle=7,delta_r=0.03):
    
        
    rho_list_all=[]
        
        
    
    for choosen_particle,j in enumerate(particles): 


        dlugosc=[]
        for i,particle in enumerate(particles):

            vector_from_choosen=particles[choosen_particle].r-particle.r

            if vector_from_choosen[0]>boxsize2:
                vector_from_choosen[0]=vector_from_choosen[0]-boxsize
            elif vector_from_choosen[0]<-boxsize2:
                vector_from_choosen[0]=vector_from_choosen[0]+boxsize

            if vector_from_choosen[1]>boxsize2:
                 vector_from_choosen[1]=vector_from_choosen[1]-boxsize
            elif vector_from_choosen[1]<-boxsize2:
                vector_from_choosen[1]=vector_from_choosen[1]+boxsize

            dlugosc.append(np.linalg.norm(vector_from_choosen))


    #     print(dlugosc)
        rho_list=[]
        r_list=[]
        r=delta_r
        while r<3.5:
            n=0
            for i in dlugosc:
                if i > r and i< r +delta_r:
                    n+=1
            rho_list.append((1/density)*n/(2*np.pi*r*delta_r))
    #         print(n)
            r_list.append(r)
            r+=delta_r
        rho_list_all.append([rho_list])
    rho_list_all=np.sum(rho_list_all,axis=0)/particleNumber
    
    return rho_list_all[0],r_list

In [19]:
en=0 #numer klatki
t=0
Cisnienie_list =[]
Temperatura_list =[]
Moment_in_time=[]
Energia_kinetyczna_list=[]
Energia_potencjalna_list=[]
Prędkości_list=[]
rho_l=0
r_l=0
rang=70000
how_many_sum=0
for itrations in tqdm(range(rang)):
    
    forces=Suma_sil(particles)
    
    frog(particles,forces)
    #implementacja perjodycznych warunków brzegowych
    for p in particles:
        if p.r[1] >= boxsize:
            p.r[1] -= boxsize
        if p.r[1] <= 0:
            p.r[1] += boxsize

        if p.r[0] >= boxsize:
            p.r[0] -= boxsize
        if p.r[0] <= 0:
            p.r[0] += boxsize
    
#     T=Temperatura(particles)
#     P=Cisnienie(particles)
#     Ek=Energia_kinetyczna(particles)
#     Ep=Energia_potencjalna(particles)
    
#     Energia_potencjalna_list.append(Ep)
#     Energia_kinetyczna_list.append(Ek)
#     Temperatura_list.append(T)
#     Cisnienie_list.append(P)
#     Moment_in_time.append(t)
    
    t+=deltat
    
    
    if (itrations >(rang-10000)):#20000
        rho,r=Distance_from_particle(particles,choosen_particle=1)
        rho_l+=np.asarray(rho)
        
        r_l=np.asarray(r)
        how_many_sum+=1

    if (en % 1000 == 0):

            plt.clf()  # czysczenie obrazka
            
            F = plt.gcf()  # zdefiniuj nowy
            for i in range(particleNumber):
                
                p = particles[i]
                color = 'b'
                a = plt.gca()
#                 if (i == 7):      #Do śledzenia położenia konkretnej cząstki
#                     color = 'r'
                color = plt.cm.summer(np.linalg.norm(p.v_plus)/temp_termostat)
                cir = Circle((p.r[0], p.r[1]), radius=p.rad, color=color)
                a.add_patch(cir)
                plt.plot()
            plt.xlim((0, boxsize))
            plt.ylim((0, boxsize))
            F.set_size_inches((6, 6))#ustawić na 6,6
            nStr = str(en)
            nStr = nStr.rjust(6, '0')
            plt.title("Symulacja gazu Lennarda-Jonesa dla temperatury"+"%.2f" % temp_termostat)#, krok " + nStr)
            
            
            
            
            plt.savefig('img' + nStr + '.png')
            plt.clf()  # czysczenie obrazka
            
    en += 1
rho_l=rho_l/how_many_sum

plt.scatter(r_l,rho_l)
# plt.show()
plt.savefig("%.2f.png" % temp_termostat)
plt.clf()


#ochładzanie
rang=40000



for temp in np.linspace(temp_termostat,temp_termostat/2.,num=5):
    temp_termostat=temp
    rho_l=0
    r_l=0
    how_many_sum=0
    
    
    for itrations in tqdm(range(rang)):
    
        forces=Suma_sil(particles)

        frog(particles,forces)
        #implementacja perjodycznych warunków brzegowych
        for p in particles:
            if p.r[1] >= boxsize:
                p.r[1] -= boxsize
            if p.r[1] <= 0:
                p.r[1] += boxsize

            if p.r[0] >= boxsize:
                p.r[0] -= boxsize
            if p.r[0] <= 0:
                p.r[0] += boxsize

    #     T=Temperatura(particles)
    #     P=Cisnienie(particles)
    #     Ek=Energia_kinetyczna(particles)
    #     Ep=Energia_potencjalna(particles)

    #     Energia_potencjalna_list.append(Ep)
    #     Energia_kinetyczna_list.append(Ek)
    #     Temperatura_list.append(T)
    #     Cisnienie_list.append(P)
    #     Moment_in_time.append(t)

        t+=deltat


        if (itrations >(rang-10000)):#20000
            rho,r=Distance_from_particle(particles,choosen_particle=1)
            rho_l+=np.asarray(rho)

            r_l=np.asarray(r)
            how_many_sum+=1

        if (en % 1000 == 0):

                plt.clf()  # czysczenie obrazka

                F = plt.gcf()  # zdefiniuj nowy
                for i in range(particleNumber):

                    p = particles[i]
                    color = 'b'
                    a = plt.gca()
    #                 if (i == 7):      #Do śledzenia położenia konkretnej cząstki
    #                     color = 'r'
                    color = plt.cm.summer(np.linalg.norm(p.v_plus)/temp_termostat)
                    cir = Circle((p.r[0], p.r[1]), radius=p.rad, color=color)
                    a.add_patch(cir)
                    plt.plot()
                plt.xlim((0, boxsize))
                plt.ylim((0, boxsize))
                F.set_size_inches((6, 6))#ustawić na 6,6
                nStr = str(en)
                nStr = nStr.rjust(6, '0')
                plt.title("Symulacja gazu Lennarda-Jonesa dla temperatury"+"%.2f" % temp_termostat)#, krok " + nStr)




                plt.savefig('img' + nStr + '.png')
                plt.clf()  # czysczenie obrazka

        en += 1
    rho_l=rho_l/how_many_sum

    plt.scatter(r_l,rho_l)
    # plt.show()
    plt.savefig("%.2f.png" % temp_termostat)
    plt.clf()

100%|██████████| 40000/40000 [56:40<00:00, 11.76it/s]


In [20]:
#ochładzanie
rang=40000



for temp in np.linspace(temp_termostat,0.01,num=5):
    temp_termostat=temp
    rho_l=0
    r_l=0
    how_many_sum=0
    
    
    for itrations in tqdm(range(rang)):
    
        forces=Suma_sil(particles)

        frog(particles,forces)
        #implementacja perjodycznych warunków brzegowych
        for p in particles:
            if p.r[1] >= boxsize:
                p.r[1] -= boxsize
            if p.r[1] <= 0:
                p.r[1] += boxsize

            if p.r[0] >= boxsize:
                p.r[0] -= boxsize
            if p.r[0] <= 0:
                p.r[0] += boxsize

    #     T=Temperatura(particles)
    #     P=Cisnienie(particles)
    #     Ek=Energia_kinetyczna(particles)
    #     Ep=Energia_potencjalna(particles)

    #     Energia_potencjalna_list.append(Ep)
    #     Energia_kinetyczna_list.append(Ek)
    #     Temperatura_list.append(T)
    #     Cisnienie_list.append(P)
    #     Moment_in_time.append(t)

        t+=deltat


        if (itrations >(rang-10000)):#20000
            rho,r=Distance_from_particle(particles,choosen_particle=1)
            rho_l+=np.asarray(rho)

            r_l=np.asarray(r)
            how_many_sum+=1

        if (en % 1000 == 0):

                plt.clf()  # czysczenie obrazka

                F = plt.gcf()  # zdefiniuj nowy
                for i in range(particleNumber):

                    p = particles[i]
                    color = 'b'
                    a = plt.gca()
    #                 if (i == 7):      #Do śledzenia położenia konkretnej cząstki
    #                     color = 'r'
                    color = plt.cm.summer(np.linalg.norm(p.v_plus)/temp_termostat)
                    cir = Circle((p.r[0], p.r[1]), radius=p.rad, color=color)
                    a.add_patch(cir)
                    plt.plot()
                plt.xlim((0, boxsize))
                plt.ylim((0, boxsize))
                F.set_size_inches((6, 6))#ustawić na 6,6
                nStr = str(en)
                nStr = nStr.rjust(6, '0')
                plt.title("Symulacja gazu Lennarda-Jonesa dla temperatury"+"%.2f" % temp_termostat)#, krok " + nStr)




                plt.savefig('img' + nStr + '.png')
                plt.clf()  # czysczenie obrazka

        en += 1
    rho_l=rho_l/how_many_sum

    plt.scatter(r_l,rho_l)
    # plt.show()
    plt.savefig("%.2f.png" % temp_termostat)
    plt.clf()

100%|██████████| 40000/40000 [56:23<00:00, 11.82it/s]


In [0]:
!ls

In [0]:
!zip -r sym.zip /content/

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
uploaded = drive.CreateFile({'title': 'sym.zip'})
uploaded.SetContentFile('sym.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
plt.figure()
plt.xlabel('Czas [s]')
plt.suptitle('Gestosć = %s' %density)
plt.ylabel('Temperatura [K]')
plt.scatter(Moment_in_time,Temperatura_list,s=.1)
plt.xlabel('Czas[s]')
# plt.tight_layout()
# plt.subplots_adjust(left=0.2, wspace=0.8, top=0.8)
plt.savefig('czas_temperatura')
plt.show()

In [0]:
plt.figure()
plt.xlabel('Czas [s]')
plt.ylabel('Cisnienie [a.u]')
plt.plot(Moment_in_time,Cisnienie_list,lw=1)
plt.xlabel('Czas[s]')
plt.tight_layout()
plt.subplots_adjust(left=0.2, wspace=0.8, top=0.8)
plt.savefig('czas_cisnienie.png')
plt.show()

In [0]:
Energia_calkowita_list=np.add(Energia_kinetyczna_list,Energia_potencjalna_list)
len(Energia_calkowita_list)

In [0]:
plt.figure(figsize=[10,11])
plt.plot(Moment_in_time,Energia_kinetyczna_list,lw=1,label='Energia kinetyczna')


plt.plot(Moment_in_time,Energia_potencjalna_list,lw=1,label='Energia potencjalna')
plt.ylabel('Energia [a.u]')
plt.xlabel('Czas [s]')
plt.plot(Moment_in_time,Energia_calkowita_list,lw=1,label='Energia calkowita')
plt.legend(bbox_to_anchor=(1., 1.05))
plt.savefig('energie.png')
plt.show()
